# CAM - class activation map

# Define models

In [1]:
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from google.colab import files

def plot_and_save(model_train, model_name):
  fig, (loss_ax, acc_ax) = plt.subplots(1, 2, figsize=(20,7))
  loss_ax.set_title('Loss')
  loss_ax.plot(model_train.history['loss'], '-r', label='Train')
  loss_ax.plot(model_train.history['val_loss'], '-g', label='Validation')
  acc_ax.set_title('Accuracy')
  acc_ax.plot(model_train.history['accuracy'], '-r', label='Train')
  acc_ax.plot(model_train.history['val_accuracy'], '-g', label='Validation')
  plt.legend(loc = 4)
  figname = "{}.png".format(model_name)
  plt.savefig(figname)
  files.download(figname)


def test_train(model, model_name: str, model_description: str):
  model_weights = "{}.h5".format(model_name)
  earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 10, verbose = 1)
  checkpoint = tf.keras.callbacks.ModelCheckpoint(model_weights, 
                                                  monitor         = 'val_accuracy', 
                                                  verbose         = 1, 
                                                  save_best_only  = True)
  start = time.time()
  model_train = model.fit(mnist_train_x, 
                          mnist_train_y, 
                          validation_split  = 0.2, 
                          callbacks         = [earlystop, checkpoint], 
                          epochs            = 10000, 
                          batch_size        = 256)
  elapsed = time.time() - start
  model.load_weights(model_weights)
  plot_and_save(model_train, model_name)
  print(model_name)
  _, train_acc = model.evaluate(mnist_train_x, mnist_train_y)
  print('Train accuracy: {}'.format(train_acc))
  _, test_acc = model.evaluate(mnist_test_x, mnist_test_y)
  print('Test accuracy: {}'.format(test_acc))
  files.download(model_weights)
  with open("results.csv", "a") as f:
    f.write("{};{};{};{};{}\n".format(model_name, model_description, elapsed, train_acc, test_acc))
  confusion_train, confusion_test = "{}-confusion_train.csv".format(model_name), "{}-confusion_test.csv".format(model_name)
  np.savetxt(confusion_train, confusion_matrix(mnist_train_y, np.argmax(model.predict(mnist_train_x),axis=1)), delimiter=",")
  np.savetxt(confusion_test, confusion_matrix(mnist_test_y, np.argmax(model.predict(mnist_test_x),axis=1)), delimiter=",")
  files.download(confusion_train)
  files.download(confusion_test)
  print("----------------------------------------------------------------------")

def model_summary(model):
  string_list = []
  model.summary(print_fn=lambda x: string_list.append(x))
  short_model_summary = "\n".join(string_list)
  return short_model_summary

In [2]:
import tensorflow_datasets as tfds

mnist_data, mnist_info = tfds.load('mnist', with_info=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
import numpy as np

mnist_train_x = np.asarray([instance['image']/255 for instance in tfds.as_numpy(mnist_data['train'])])
mnist_train_y = np.asarray([instance['label'] for instance in tfds.as_numpy(mnist_data['train'])])

mnist_test_x = np.asarray([instance['image']/255 for instance in tfds.as_numpy(mnist_data['test'])])
mnist_test_y = np.asarray([instance['label'] for instance in tfds.as_numpy(mnist_data['test'])])

In [4]:
image_shape = mnist_train_x[0].shape
image_shape

(28, 28, 1)

In [6]:
import tensorflow as tf

model_name_c2 = "model_c2"
model_description_c2 =\
'''
SAPO
'''
model_c2 = tf.keras.Sequential(name = model_name_c2)
model_c2.add(tf.keras.layers.Input(image_shape))
model_c2.add(tf.keras.layers.Conv2D(filters = 16, 
                                    kernel_size = (3,3),
                                    activation="relu", 
                                    padding="same", 
                                    name="convolution_1"))
model_c2.add(tf.keras.layers.Conv2D(filters = 16, 
                                    kernel_size = (3,3),
                                    activation="relu", 
                                    padding="same", 
                                    name="convolution_2"))
model_c2.add(tf.keras.layers.Conv2D(filters = 32, 
                                    kernel_size = (3,3),
                                    activation="relu", 
                                    padding="same", 
                                    name="convolution_3"))
model_c2.add(tf.keras.layers.Conv2D(filters = 256, 
                                    kernel_size = (3,3),
                                    activation="relu", 
                                    padding="same", 
                                    name="convolution_4"))
model_c2.add(tf.keras.layers.GlobalAveragePooling2D(name = "GAP"))
model_c2.add(tf.keras.layers.Dense(10, 
                                   activation='softmax', 
                                   name='output',
                                   use_bias = False))
model_c2.compile(loss='sparse_categorical_crossentropy', 
                 optimizer='Adam', 
                 metrics=['accuracy'])
model_c2.summary()
model_description_c2 = model_summary(model_c2) + model_description_c2
model_description_c2 = model_description_c2.replace("\n","\\n")

Model: "model_c2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution_1 (Conv2D)       (None, 28, 28, 16)        160       
_________________________________________________________________
convolution_2 (Conv2D)       (None, 28, 28, 16)        2320      
_________________________________________________________________
convolution_3 (Conv2D)       (None, 28, 28, 32)        4640      
_________________________________________________________________
convolution_4 (Conv2D)       (None, 28, 28, 256)       73984     
_________________________________________________________________
GAP (GlobalAveragePooling2D) (None, 256)               0         
_________________________________________________________________
output (Dense)               (None, 10)                2560      
Total params: 83,664
Trainable params: 83,664
Non-trainable params: 0
______________________________________________________

## Define functions

## Test and train

In [ ]:
with open("results.csv", "w+") as f:
  f.write("model name;model description;training time;train accuracy;test accuracy\n")
test_train(model_c2, model_name_c2, model_description_c2)
files.download("results.csv")

Epoch 1/10000
188/188 [==============================] - 448s 2s/step - loss: 1.7468 - accuracy: 0.3492 - val_loss: 1.3548 - val_accuracy: 0.5152

Epoch 00001: val_accuracy improved from -inf to 0.51517, saving model to model_c2.h5
Epoch 2/10000
188/188 [==============================] - 450s 2s/step - loss: 1.1242 - accuracy: 0.6369 - val_loss: 0.9341 - val_accuracy: 0.7237

Epoch 00002: val_accuracy improved from 0.51517 to 0.72367, saving model to model_c2.h5
Epoch 3/10000
188/188 [==============================] - 449s 2s/step - loss: 0.7685 - accuracy: 0.7657 - val_loss: 0.6592 - val_accuracy: 0.8069

Epoch 00003: val_accuracy improved from 0.72367 to 0.80692, saving model to model_c2.h5
Epoch 4/10000
188/188 [==============================] - 449s 2s/step - loss: 0.5844 - accuracy: 0.8252 - val_loss: 0.5165 - val_accuracy: 0.8444

Epoch 00004: val_accuracy improved from 0.80692 to 0.84442, saving model to model_c2.h5
Epoch 5/10000
188/188 [==============================] - 450s 2